In [ ]:
import random
random.seed(0)

import os 
from importlib import reload
import functions as fct
reload(fct)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import metric_seq2seq
reload(metric_seq2seq)
from metric_seq2seq import PrecisionSeq, RecallSeq, AUCSeq, F1ScoreSeq

ind_mf4_train = 4

ind_mic = 1
sr = 100000
tfn = 0.02
FMAX = 22000
seq2seq = False
N_window = fct.get_near_pof2(tfn,sr)
NB_FEATURES = 64
apply_pca = False
    #extraire les coefs mels
if True :  
    mels ,labels = [],[]
    for ind_train,debut,fin in zip([ind_mf4_train],[int(1*60)],[int(17*60+40)]):
        xi,yi = fct.fft_features(list_mf4 = [ind_train],
                                    list_mics = [ind_mic],
                                    N_window = N_window,
                                    #nb_mels = NB_FEATURES,
                                    #power = 2.0,
                                    debut = debut,
                                    fin = fin,
                                    annot = True,
                                    sous_mean = True,
                                    feature = "audio",
                                    f_max = FMAX,
                                    overlap = 2, #25% de hop_lenght pour le calule des mels
                                    #energie = "both",
                                    precoce = True,
                                    #derive1 = False,
                                    #derive2 = False,
                                    
                                    nbr_classes= 2 )
        mels.append(xi)
        labels.append(yi)


    X,Y = [],[]
    X_val,Y_val = [],[]
    X_test,Y_test = [],[]

    for key in mels[0].keys():
        X,Y = np.array(mels[0][key]),np.array(labels[0][key])


    #prétraitement des donnees
    X,Y,class_weights,pca,scaler = fct.pretrait(X,Y,n_components=0.95,
                                                apply_norm=True,
                                                seq=False,
                                                seq2seq=seq2seq,
                                                seq_apres=10,
                                                seq_before=9,
                                                apply_pca=True,
                                                overlap=0.5,
                                                super_vecteur=False
                                                )


    X_train,X_val,Y_train,Y_val = train_test_split(X,Y,stratify=Y,test_size=0.4,random_state=0)
    X_val,X_test,Y_val,Y_test = train_test_split(X_val,Y_val,stratify=Y_val,random_state=0,test_size=0.5)

weights = np.ones(Y_train.shape[0], dtype = 'float')
for i, val in enumerate(Y_train):
    weights[i] = class_weights[val-1]

Normalisation par rapport au dataset est appliquée aux données d'entrainemnet
PCA entraînée et appliquée aux données d'entrainement - Nouvelle forme: (97655, 439)
Poids de classe calculés [ 0.50433817 58.12797619]


In [8]:
Xt = X_train[:,:]
Xv = X_val[:,:]
print(Xt.shape)
print(np.bincount(Y_train))

(58593, 439)
[58089   504]


In [ ]:
import optuna
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, test_size=0.2, random_state=0)

# Création du vecteur de poids
weights = np.ones(Y_train.shape[0], dtype='float')
for i, val in enumerate(Y_train):
    weights[i] = class_weights[val - 1]

# Fonction objectif
def objective(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'n_estimators': trial.suggest_categorical('n_estimators', [50,100,500]),
        'max_depth': trial.suggest_categorical('max_depth', [3,5,7,9,11]),
        'subsample': trial.suggest_categorical('subsample', [0.5,0.7,1.0]),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.5,0.7,1]),
        'eval_metric': 'logloss',
        'random_state': 0
    }

    model = XGBClassifier(**params)
    model.fit(X_train, Y_train, sample_weight=weights)
    y_pred = model.predict(X_test)
    f1_class1 = f1_score(Y_test, y_pred, pos_label=1)

    # Affichage pour chaque trial
    print(f"Trial {trial.number} - F1 (classe 1): {f1_class1:.4f} | Params: {params}")

    return f1_class1

# Lancer l’étude Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)


In [9]:
import librosa
import soundfile as sf
import os

def decouper_audio(path_in, path_out, start_sec, end_sec):
    """
    Découpe un extrait d'un fichier audio et l'enregistre dans un nouveau fichier.
    
    :param path_in: chemin du fichier audio d'entrée (.wav, .mp3, etc.)
    :param path_out: chemin du nouveau fichier audio de sortie (.wav de préférence)
    :param start_sec: début de l'extrait en secondes
    :param end_sec: fin de l'extrait en secondes
    """
    # Charger l'audio (en mono)
    y, sr = librosa.load(path_in, sr=100000)
    
    # Convertir secondes en indices
    start_sample = int(start_sec * sr)
    end_sample = int(end_sec * sr)
    
    # Vérification des bornes
    if start_sample < 0 or end_sample > len(y) or start_sample >= end_sample:
        raise ValueError("Plage temporelle invalide.")
    
    # Extraire l'extrait
    extrait = y[start_sample:end_sample]
    
    # Sauvegarder dans un nouveau fichier WAV
    sf.write(path_out, extrait, sr)
    print(f"✅ Extrait sauvegardé dans {path_out}")

path_in =  os.path.join("data","audios","Son_3_4.wav")          #"C:\\Users\\mohammed-amine.tanna\\OneDrive - RAILENIUM\\Bureau\\stage_railenium\\Telli\\Revigny\\data\\audios\\Son_4_1.wav"
path_out = "extrait_3_4_33.wav"
start_sec = int(2*60+35)               #int(6*60+7)           int(7*60 + 37)   
end_sec =   int(9*60+24)              #int(8*60+10)          int(10*60+37)
decouper_audio(path_in=path_in,path_out=path_out,start_sec=start_sec,end_sec=end_sec)   #5 joints

✅ Extrait sauvegardé dans extrait_3_4_33.wav
